In [ ]:
# print('Program started')
import vrep
import numpy as np
import cv2
import cv2.aruco as aruco
import sys, time, math
from platform import python_version

##############Comunicação com V-REP##########################
serverIP = '127.0.0.1';
serverPort = 19999; #Esta porta do servidor está sempre aberta
vrep.simxFinish(-1);
clientID=vrep.simxStart(serverIP,serverPort,True,True,5000,5);
#############################################################

###############################################################################
#------- ROTATIONS https://www.learnopencv.com/rotation-matrix-to-euler-angles/

# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R):
    assert (isRotationMatrix(R))

    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z])

###############################################################################

#-- Update fps
def update_fps_read():
    global t_read, fps_read
    t           = time.time()
    fps_read    = 1.0/(t - t_read)
    t_read      = t
    
def update_fps_detect():
    global t_detect, fps_detect
    t           = time.time()
    fps_detect  = 1.0/(t - t_detect)
    t_detect      = t
    
t_read      = time.time()
t_detect    = t_read
fps_read    = 0.0
fps_detect  = 0.0
    
###############################################################################

windowName = "Imagem-Processada" #Name of the window created
#cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)#Setting the name ande type of window
#cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_KEEPRATIO)#setting fullscreen

#-- Define Tag
id_to_find = 2
marker_size = 100 #-cm

#-- Get the camera calibration
calib_path = ''
camera_matrix = np.loadtxt(calib_path+'cameraMatrix.txt', delimiter = ',')
camera_distortion = np.loadtxt(calib_path+'cameraDistortion.txt', delimiter = ',')

#-- 180 deg rotation matrix around x axis
R_flip = np.zeros((3,3), dtype=np.float)
R_flip[0,0] = 1.0
R_flip[1,1] = -1.0
R_flip[2,2] = -1.0

#-- variables of control
ti=0
k=0.08 #constante de tempo
ki=0.01
ep = 0 #erro passado  = 0

#-- Font for the text in the image
font = cv2.FONT_HERSHEY_PLAIN

#-- Define the Aruco dictionary
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_50)
parameters =  aruco.DetectorParameters_create()

if clientID!=-1:
    print ('Connected to remote API server')
    print ('Versão OpenCV: ',cv2.__version__)
    print ('Versão Python: ',python_version())
    
    err,visionHandle = vrep.simxGetObjectHandle(clientID,'Vision_sensor',vrep.simx_opmode_oneshot_wait)
    
    while err == 0:
        
        update_fps_read()
        
        err,visionHandle = vrep.simxGetObjectHandle(clientID,'Vision_sensor',vrep.simx_opmode_oneshot_wait)
        err,tgHandle=vrep.simxGetObjectHandle(clientID,'Quadricopter_target',vrep.simx_opmode_oneshot_wait)
        
        err, pos = vrep.simxGetObjectPosition(clientID,tgHandle,tgHandle,vrep.simx_opmode_oneshot_wait) #Posição do objeto a ser seguido pelo Drone.
        err, ori = vrep.simxGetObjectOrientation(clientID,tgHandle,-1,vrep.simx_opmode_oneshot_wait) #Orientação do objeto a ser seguido pelo Drone.
        
        err,res,imgList = vrep.simxGetVisionSensorImage(clientID,visionHandle,0,vrep.simx_opmode_oneshot_wait) #Imagens do sensor
        
        #-- Convert the list of image to array and invert to get a good plot
        img = (np.array(imgList))[::-1]#Passa de lista para uma array e inverte a array para plotar corretamente
        
        #-- Segment the image in one image of size 1280x720 or 640x480
        imgRGB = cv2.flip(np.uint8(img.reshape(512,512,3)),1) #Transforma em uma imagem RGB e espelha
        
        #-- Convert in gray scale
        gray = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2GRAY) #-- remember, OpenCV stores color images in Blue, Green, Red
        
        #-- Find all the aruco markers in the image
        corners, ids, rejected = aruco.detectMarkers(image=gray, 
                                                     dictionary=aruco_dict, 
                                                     parameters=parameters,
                                                     cameraMatrix=camera_matrix, 
                                                     distCoeff=camera_distortion)
        
        #-- If there is a id and compare if id is equal to the searched
        if ids != None and ids[0] == id_to_find:
            
            update_fps_detect()
            
            #-- ret= [rvec,tvec, ?]
            #-- array of rotation and position of each marker in camera frame
            #-- rvec = [[rvec_1, [rvec2], ...]]  attitude of the marker respect to camera frame
            #-- tvec = [[tvec_1, [tvec2], ...]]  position of the marker in camera frame
            ret = aruco.estimatePoseSingleMarkers(corners, marker_size, camera_matrix, camera_distortion)

            #-- Unpack the output, get only the first
            rvec, tvec = ret[0][0,0,:], ret[1][0,0,:]

            #-- Draw the detected marker and put a reference frame over it
            aruco.drawDetectedMarkers(imgRGB, corners)
            aruco.drawAxis(imgRGB, camera_matrix, camera_distortion, rvec, tvec, 40)
            
            #-- Obtain the rotation matrix tag->camera
            R_ct = np.matrix(cv2.Rodrigues(rvec)[0])
            R_tc = R_ct.T # function transpose() with '.T'
            #print("rotação transposta:",R_tc)
            
            #-- Get the attitude in terms of euler 321 (Needs to be flipped first)
            roll_marker, pitch_marker, yaw_marker = rotationMatrixToEulerAngles(R_tc)
            
            #-- Now get Position and attitude f the camera respect to the marker
            pos_camera = -R_tc*np.matrix(tvec).T
            roll_camera, pitch_camera, yaw_camera = rotationMatrixToEulerAngles(R_flip*R_tc)
            
            ###############################################################################
            #-- Print the tag position in camera frame
            str_position = "MARKER Position x=%4.0f  y=%4.0f  z=%4.0f "%(tvec[0], tvec[1], tvec[2])
            cv2.putText(imgRGB, str_position, (0, 50), font, 1, (255, 255, 0), 1, cv2.LINE_AA)
            
            #-- Print the marker's attitude respect to camera frame
            str_attitude = "MARKER Attitude r=%4.0f  p=%4.0f  y=%4.0f"%(math.degrees(roll_marker),math.degrees(pitch_marker),
                                math.degrees(yaw_marker))
            cv2.putText(imgRGB, str_attitude, (0, 100), font, 1, (255, 255, 0), 1, cv2.LINE_AA)
            
            
            #-- Print the tag position in camera frame
            str_position = "CAMERA Position x=%4.0f  y=%4.0f  z=%4.0f"%(pos_camera[0], pos_camera[1], pos_camera[2])
            cv2.putText(imgRGB, str_position, (0, 150), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

            #-- Get the attitude of the camera respect to the frame
            str_attitude = "CAMERA Attitude r=%4.0f  p=%4.0f  y=%4.0f"%(math.degrees(roll_camera),math.degrees(pitch_camera),
                                math.degrees(yaw_camera))
            cv2.putText(imgRGB, str_attitude, (0, 200), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
            
            ###############################################################################

            #-- Display the resulting frame
            cv2.imshow(windowName,imgRGB)
            
            # print 'Camera X = %.1f  Y = %.1f  Z = %.1f  - fps = %.0f'%(pos_camera[0], pos_camera[1], pos_camera[2],fps_detect)
            # print('Marker X = {:.2f}  Y = {:.2f}  Z = {:.2f}  - fps = {:.2f}'.format(tvec[0], tvec[1], tvec[2],fps_detect))
            # print('pos - X = {:.2f}  Y = {:.2f}  Z = {:.2f}'.format(pos[0], pos[1], pos[2]))
            
            
            #print('fps = {:.2f}'.format(fps_read))
            
            #ajust = (math.degrees(yaw_camera)*(1-math.exp(-ti/k)))
            #dyaw = math.degrees(yaw_camera) - ajust
            #ti = ti + 1
            
            #print('dyaw = {} - ajust = {} - t = {}'.format(dyaw/2,ajust,ti))
            
            e = math.degrees(yaw_camera)
            u = k*e+(e+ep)*ki
            ep = e
            print('u = {}'.format(u))
            
            #if dyaw>0:
            tgori=[ori[0],ori[1],ori[2]+(u*np.pi/180)]
            vrep.simxSetObjectOrientation(clientID,tgHandle,-1,tgori,vrep.simx_opmode_oneshot)
            
            
            #elif dyaw<0:
                #tgori=[ori[0],ori[1],ori[2]-(dyaw*np.pi/180)]
                #vrep.simxSetObjectOrientation(clientID,tgHandle,-1,tgori,vrep.simx_opmode_oneshot)
            
            #-- correction of angle yaw
#             if math.degrees(yaw_camera)>=10:
#                 tgori=[ori[0],ori[1],ori[2]+(2*np.pi/180)]
#                 vrep.simxSetObjectOrientation(clientID,tgHandle,-1,tgori,vrep.simx_opmode_oneshot)
#             elif math.degrees(yaw_camera)>=1 and math.degrees(yaw_camera)<10:
#                 tgori=[ori[0],ori[1],ori[2]+(0.3*np.pi/180)]
#                 vrep.simxSetObjectOrientation(clientID,tgHandle,-1,tgori,vrep.simx_opmode_oneshot)
#                #print('ori - {} '.format(tgori))
#             elif math.degrees(yaw_camera)<=-10:
#                 tgori=[ori[0],ori[1],ori[2]-(2*np.pi/180)]
#                 vrep.simxSetObjectOrientation(clientID,tgHandle,-1,tgori,vrep.simx_opmode_oneshot)
#             elif math.degrees(yaw_camera)<=-1 and math.degrees(yaw_camera)>-10:
#                 tgori=[ori[0],ori[1],ori[2]-(0.3*np.pi/180)]
#                 vrep.simxSetObjectOrientation(clientID,tgHandle,-1,tgori,vrep.simx_opmode_oneshot)
                
            #-- correction of position x
            #if pos_camera[0]>=10:
            #    tgpos=[pos[0]-1,pos[1],pos[2]]# Movimenta para a esquerda
            #    vrep.simxSetObjectPosition(clientID,tgHandle,tgHandle,tgpos,vrep.simx_opmode_oneshot)
            #elif pos_camera[0]>=1 and pos_camera[0]<10:
            #    tgpos=[pos[0]-0.1,pos[1],pos[2]]# Movimenta para a esquerda
            #    vrep.simxSetObjectPosition(clientID,tgHandle,tgHandle,tgpos,vrep.simx_opmode_oneshot)
            #elif math.degrees(yaw_camera)<=-10:
            #    tgpos=[pos[0]+1,pos[1],pos[2]]# Movimenta para a esquerda
            #    vrep.simxSetObjectPosition(clientID,tgHandle,tgHandle,tgpos,vrep.simx_opmode_oneshot)
            #elif math.degrees(yaw_camera)<=-1 and math.degrees(yaw_camera)>-10:
            #    tgpos=[pos[0]+1,pos[1],pos[2]]# Movimenta para a esquerda
            #    vrep.simxSetObjectPosition(clientID,tgHandle,tgHandle,tgpos,vrep.simx_opmode_oneshot)
            
            #-- Exit on ESC
            if cv2.waitKey(1) == 27:
                break
        
        else:
            print('Nothing detected - fps = {:.2f}'.format(fps_read))
            
            #-- Display the resulting frame
            cv2.imshow(windowName,imgRGB)
            #-- Exit on ESC
            if cv2.waitKey(1) == 27:
                break

    #-- Now close the connection to V-REP:
    vrep.simxFinish(clientID)

else:
    print ('Failed connecting to remote API server')
print ('Program ended')

#-- destroy all windows
cv2.destroyAllWindows()

Connected to remote API server
Versão OpenCV:  3.4.2
Versão Python:  3.6.5
u = 12.270063032624188
u = 13.527031762672472
u = 13.007084051704952
u = 12.108301453910467
u = 10.94322896275617
u = 9.528824691688058
u = 7.964724732884739
u = 6.400134173008746
u = 4.961074360588055
u = 3.707547712433969
u = 2.7280290544485175
u = 2.0791165104680416
u = 1.6966051489471639
u = 1.572601198717682
u = 1.7073547322657998
u = 1.9332321242241821
u = 2.2277114067906925
u = 2.4621580101957785
u = 2.695076038809141
u = 2.7707754616788813
u = 2.724249173872319
u = 2.5349898315909987
u = 2.250591800155175
u = 1.8532074593271508
u = 1.4334345645519686
u = 1.0706907822972789
u = 0.7200594034373495
u = 0.44089105697574316
u = 0.2162494922571881
u = 0.12658574475113593
u = 0.06945789303576508
u = 0.1520058657174439
u = 0.19105238853356757
u = 0.3153656830402584
u = 0.43731600621480815
u = 0.5210475358391587
u = 0.6025989739270153
u = 0.654471308972836
u = 0.6156304608898246
u = 0.5371879263075526
u = 0.46548